<h1>Appendix A -- Advanced NumPy</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

<p>In this appendix, I will go deeper into the NumPy library for array computing. This will include more internal details about the ndarray type and more advanced array manipulations and algorithms.</p>

<p>This appendix contains miscellaneous topics and does not necessarily need to be read linearly. Throughout the chapters, I will generate random data for many examples that will use the default random number generator in the <code>numpy.random</code> module:</p>

In [2]:
rng = np.random.default_rng(seed=12345)

<h2>A.1 ndarray Object Internals</h2>

<p>The NumPy ndarray provides a way to interpret a block of homogeneously typed data (either contiguous or strided) as a multidimensional array object. The data type, or <em>dtype</em>, determines how the data is interpreted as being floating point, integer, Boolean, or any of the other types we’ve been looking at.</p>

<p>Part of what makes ndarray flexible is that every array object is a strided view on a block of data. You might wonder, for example, how the array view <code>arr[::2, ::-1]</code> does not copy any data. The reason is that the ndarray is more than just a chunk of memory and a data type; it also has <em>striding</em> information that enables the array to move through memory with varying step sizes. More precisely, the ndarray internally consists of the following:</p>
<ul>
    <li>A <em>pointer to data</em>—that is, a block of data in RAM or in a memory-mapped file</li>
    <li>The <em>data type</em> or dtype describing fixed-size value cells in the array</li>
    <li>A tuple indicating the array’s <em>shape</em></li>
    <li>A tuple of strides—integers indicating the number of bytes to “step” in order to advance one element along a dimension</li>
</ul>
<p>See <a href="#numpy_ndarray">Fig. A.1: The NumPy ndarray object </a> for a simple mock-up of the ndarray innards.</p>
<figure id='numpy_ndarray'>
    <img src="images/pda3_a001.png">
    <figcaption>Fig. A.1: The NumPy ndarray object</figcaption>
</figure>
<p>For example, a 10 × 5 array would have the shape <code>(10, 5)</code>:</p>


In [3]:
np.ones((10, 5)).shape

(10, 5)

<p>A typical (C order) 3 × 4 × 5 array of <code>float64</code> (8-byte) values has the strides <code>(160, 40, 8)</code> (knowing about the strides can be useful because, in general, the larger the strides on a particular axis, the more costly it is to perform computation along that axis):</p>

In [4]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

<p>While it is rare that a typical NumPy user would be interested in the array strides, they are needed to construct "zero-copy" array views. Strides can even be negative, which enables an array to move "backward" through memory (this would be the case, for example, in a slice like <code>obj[::-1]</code> or <code>obj[:, ::-1]</code>).</p>

<h2>NumPy Data Type Hierarchy</h2>

<p>You may occasionally have code that needs to check whether an array contains integers, floating-point numbers, strings, or Python objects. Because there are multiple types of floating-point numbers (<code>float16</code> through <code>float128</code>), checking that the data type is among a list of types would be very verbose. Fortunately, the data types have superclasses, such as <code>np.integer</code> and <code>np.floating</code>, which can be used with the <code>np.issubdtype</code> function:</p>



In [5]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)

True

In [6]:
np.issubdtype(floats.dtype, np.floating)

True

<p>You can see all of the parent classes of a specific data type by calling the type’s <code>mro</code> method:</p>

In [7]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

<p>Therefore, we also have:</p>

In [8]:
np.issubdtype(ints.dtype, np.number)

True

<p>Most NumPy users will never have to know about this, but it is occasionally useful. See <a href='#numpy_data_type'>Fig. A.2: The NumPy data type class hierarchy</a> for a graph of the data type hierarchy and parent–subclass relationships.</p>
<figure id='numpy_data_type'>
    <img src="images/pda3_a002.png">
    <figcaption>Fig. A.2: The NumPy data type class hierarchy</figcaption>
</figure>

<h2>A.2 Advanced Array Manipulation</h2>

<p>There are many ways to work with arrays beyond fancy indexing, slicing, and Boolean subsetting. While much of the heavy lifting for data analysis applications is handled by higher-level functions in pandas, you may at some point need to write a data algorithm that is not found in one of the existing libraries.</p>

<h3>Reshaping Arrays</h3>

<p>In many cases, you can convert an array from one shape to another without copying any data. To do this, pass a tuple indicating the new shape to the <code>reshape</code> array instance method. For example, suppose we had a one-dimensional array of values that we wished to rearrange into a matrix (this is illustrated in <a href="#reshaping">Fig. A.3: Reshaping in C (row major) or FORTRAN (column major) order</a>):</p>

In [9]:
arr = np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [10]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

<figure id='reshaping'>
    <img src="images/pda3_a003.png">
    <figcaption>Fig. A.3: Reshaping in C (row major) or FORTRAN (column major> order</figcaption>
</figure>
<p>A multidimensional array can also be reshaped:</p>

In [11]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

<p>One of the passed shape dimensions can be –1, in which case the value used for that dimension will be inferred from the data:</p>

In [12]:
arr = np.arange(15)
arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

<p>Since an array’s <code>shape</code> attribute is a tuple, it can be passed to <code>reshape</code>, too:</p>

In [13]:
other_arr = np.ones((3, 5))
other_arr.shape

(3, 5)

In [14]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

<p>The opposite operation of <code>reshape</code> from one-dimensional to a higher dimension is typically known as <em>flattening</em> or <em>raveling</em>:</p>

In [15]:
arr = np.arange(15).reshape((5, 3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [16]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

<p><code>ravel</code> does not produce a copy of the underlying values if the values in the result were contiguous in the original array.</p>

<p>The <code>flatten</code> method behaves like <code>ravel</code> except it always returns a copy of the data:</p>

In [17]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

<p>The data can be reshaped or raveled in different orders. This is a slightly nuanced topic for new NumPy users and is therefore the next subtopic.</p>

<h3>C Versus FORTRAN Order</h3>

<p>NumPy is able to adapt to many different layouts of your data in memory. By default, NumPy arrays are created in <em>row major</em> order. Spatially this means that if you have a two-dimensional array of data, the items in each row of the array are stored in adjacent memory locations. The alternative to row major ordering is <em>column major</em> order, which means that values within each column of data are stored in adjacent memory locations.</p>

<p>For historical reasons, row and column major order are also known as C and FORTRAN order, respectively. In the FORTRAN 77 language, matrices are all column major.</p>

<p>Functions like <code>reshape</code> and <code>ravel</code> accept an order argument indicating the order to use the data in the array. This is usually set to <code>'C'</code> or <code>'F'</code> in most cases (there are also less commonly used options <code>'A'</code> and <code>'K'</code>; see the NumPy documentation, and refer back to <a href='#reshaping'>Fig. A.3: Reshaping in C (row major) or FORTRAN (column major) order for an illustration of these options</a>):</p>

In [18]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [20]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

<p>Reshaping arrays with more than two dimensions can be a bit mind-bending (see <a href='#reshaping'>Reshaping in C (row major) or FORTRAN (column major) order</a>). The key difference between C and FORTRAN order is the way in which the dimensions are walked:</p>
<dl>
    <dt>C/row major order</dt>
    <dd>Traverse higher dimensions first (e.g., axis 1 before advancing on axis 0).</dd>
    <dt>FORTRAN/column major order</dt>
    <dd>Traverse higher dimensions last (e.g., axis 0 before advancing on axis 1).</dd>
</dl>

<h3>Concatenating and Splitting Arrays</h3>

<p><code>numpy.concatenate</code> takes a sequence (tuple, list, etc.) of arrays and joins them in order along the input axis:</p>

In [21]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [22]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

<p>There are some convenience functions, like <code>vstack</code> and <code>hstack</code>, for common kinds of concatenation. The preceding operations could have been expressed as:</p>

In [23]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [24]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

<p><code>split</code>, on the other hand, slices an array into multiple arrays along an axis:</p>

In [25]:
arr = rng.standard_normal((5, 2))
arr

array([[-1.4238,  1.2637],
       [-0.8707, -0.2592],
       [-0.0753, -0.7409],
       [-1.3678,  0.6489],
       [ 0.3611, -1.9529]])

In [26]:
first, second, third = np.split(arr, [1, 3])
first

array([[-1.4238,  1.2637]])

In [27]:
second

array([[-0.8707, -0.2592],
       [-0.0753, -0.7409]])

In [28]:
third

array([[-1.3678,  0.6489],
       [ 0.3611, -1.9529]])

<p>The value <code>[1, 3]</code> passed to <code>np.split</code> indicates the indices at which to split the array into pieces.</p>

<p>See <a href='#array_concatenation'>Array concatenation functions</a> for a list of all relevant concatenation and splitting functions, some of which are provided only as a convenience of the very general-purpose <code>concatenate</code>.</p>
<table id='array_concatenation'>
    <caption>Table A.1: Array concatenation functions</caption>
    <tr>
        <th>Function</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>concatenate</code></td>
        <td>Most general function, concatenate collection of arrays along one axis</td>
    </tr>
    <tr>
        <td><code>vstack</code>, <code>row_stack</code></td>
        <td>Stack arrays by rows (along axis 0)</td>
    </tr>
    <tr>
        <td><code>hstack</code></td>
        <td>Stack arrays by columns (along axis 1)</td>
    </tr>
    <tr>
        <td><code>column_stack</code></td>
        <td>Like <code>hstack</code>, but convert 1D arrays to 2D column vectors first</td>
    </tr>
    <tr>
        <td><code>dstack</code></td>
        <td>Stack arrays by “depth” (along axis 2)</td>
    </tr>
    <tr>
        <td><code>split</code></td>
        <td>Split array at passed locations along a particular axis</td>
    </tr>
    <tr>
        <td><code>hsplit/vsplit</code></td>
        <td>Convenience functions for splitting on axis 0 and 1, respectively</td>
    </tr>
</table>
    
<h4>Stacking helpers: <code>r_</code> and <code>c_</code></h4>

<p>There are two special objects in the NumPy namespace, <code>r_</code> and <code>c_</code>, that make stacking arrays more concise:</p>

In [29]:
arr = np.arange(6)
arr1 = arr.reshape((3, 2))
arr2 = rng.standard_normal((3, 2))
np.r_[arr1, arr2]

array([[ 0.    ,  1.    ],
       [ 2.    ,  3.    ],
       [ 4.    ,  5.    ],
       [ 2.3474,  0.9685],
       [-0.7594,  0.9022],
       [-0.467 , -0.0607]])

In [30]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.    ,  1.    ,  0.    ],
       [ 2.    ,  3.    ,  1.    ],
       [ 4.    ,  5.    ,  2.    ],
       [ 2.3474,  0.9685,  3.    ],
       [-0.7594,  0.9022,  4.    ],
       [-0.467 , -0.0607,  5.    ]])

<p>These additionally can translate slices to arrays:</p>

In [31]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

<p>See the docstring for more on what you can do with <code>c_</code> and <code>r_</code>.</p>

<h3>Repeating Elements: <code>tile</code> and <code>repeat</code></h3>

<p>Two useful tools for repeating or replicating arrays to produce larger arrays are the <code>repeat</code> and <code>tile</code> functions. repeat replicates each element in an array some number of times, producing a larger array:</p>

In [32]:
arr = np.arange(3)
arr

array([0, 1, 2])

In [33]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

<p><strong>Note</strong><br />
    The need to replicate or repeat arrays can be less common with NumPy than it is with other array programming frameworks like MATLAB. One reason for this is that <em>broadcasting</em> often fills this need better, which is the subject of the next section.</p>

<p>By default, if you pass an integer, each element will be repeated that number of times. If you pass an array of integers, each element can be repeated a different number of times:</p>

In [34]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

<p>Multidimensional arrays can have their elements repeated along a particular axis:</p>

In [35]:
arr = rng.standard_normal((2, 2))
arr

array([[ 0.7888, -1.2567],
       [ 0.5759,  1.399 ]])

In [36]:
arr.repeat(2, axis=0)

array([[ 0.7888, -1.2567],
       [ 0.7888, -1.2567],
       [ 0.5759,  1.399 ],
       [ 0.5759,  1.399 ]])

In [37]:
arr.repeat([2, 3], axis=0)

array([[ 0.7888, -1.2567],
       [ 0.7888, -1.2567],
       [ 0.5759,  1.399 ],
       [ 0.5759,  1.399 ],
       [ 0.5759,  1.399 ]])

In [38]:
arr.repeat([2, 3], axis=1)

array([[ 0.7888,  0.7888, -1.2567, -1.2567, -1.2567],
       [ 0.5759,  0.5759,  1.399 ,  1.399 ,  1.399 ]])

<p><code>tile</code>, on the other hand, is a shortcut for stacking copies of an array along an axis. Visually you can think of it as being akin to “laying down tiles”:</p>

In [39]:
arr

array([[ 0.7888, -1.2567],
       [ 0.5759,  1.399 ]])

In [40]:
np.tile(arr, 2)

array([[ 0.7888, -1.2567,  0.7888, -1.2567],
       [ 0.5759,  1.399 ,  0.5759,  1.399 ]])

<p>The second argument is the number of tiles; with a scalar, the tiling is made row by row, rather than column by column. The second argument to <code>tile</code> can be a tuple indicating the layout of the “tiling”:</p>

In [41]:
arr

array([[ 0.7888, -1.2567],
       [ 0.5759,  1.399 ]])

In [42]:
np.tile(arr, (2, 1))

array([[ 0.7888, -1.2567],
       [ 0.5759,  1.399 ],
       [ 0.7888, -1.2567],
       [ 0.5759,  1.399 ]])

In [43]:
np.tile(arr, (3, 2))

array([[ 0.7888, -1.2567,  0.7888, -1.2567],
       [ 0.5759,  1.399 ,  0.5759,  1.399 ],
       [ 0.7888, -1.2567,  0.7888, -1.2567],
       [ 0.5759,  1.399 ,  0.5759,  1.399 ],
       [ 0.7888, -1.2567,  0.7888, -1.2567],
       [ 0.5759,  1.399 ,  0.5759,  1.399 ]])

<h3>Fancy Indexing Equivalents: take and put</h3>

<p>As you may recall from Ch 4: NumPy Basics: Arrays and Vectorized Computation, one way to get and set subsets of arrays is by <em>fancy</em> indexing using integer arrays:</p>

In [44]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

<p>There are alternative ndarray methods that are useful in the special case of making a selection only on a single axis:</p>

In [45]:
arr.take(inds)

array([700, 100, 200, 600])

In [46]:
arr.put(inds, 42)
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [47]:
arr.put(inds, [40, 41, 42, 43])
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

<p>To use <code>take</code> along other axes, you can pass the <code>axis</code> keyword:</p>

In [48]:
inds = [2, 0, 2, 1]
arr = rng.standard_normal((2, 4))
arr

array([[ 1.3223, -0.2997,  0.9029, -1.6216],
       [-0.1582,  0.4495, -1.3436, -0.0817]])

In [49]:
arr.take(inds, axis=1)

array([[ 0.9029,  1.3223,  0.9029, -0.2997],
       [-1.3436, -0.1582, -1.3436,  0.4495]])

<p><code>put</code> does not accept an axis argument but rather indexes into the flattened (one-dimensional, C order) version of the array. Thus, when you need to set elements using an index array on other axes, it is best to use <code>[]</code>-based indexing.</p>

<h2>A.3 Broadcasting</h2>

<p>Broadcasting governs how operations work between arrays of different shapes. It can be a powerful feature, but it can cause confusion, even for experienced users. The simplest example of broadcasting occurs when combining a scalar value with an array:</p>

In [50]:
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [51]:
arr * 4

array([ 0,  4,  8, 12, 16])

<p>Here we say that the scalar value 4 has been <em>broadcast</em> to all of the other elements in the multiplication operation.</p>

<p>For example, we can demean each column of an array by subtracting the column means. In this case, it is necessary only to subtract an array containing the mean of each column:</p>

In [52]:
arr = rng.standard_normal((4, 3))
arr.mean(0)

array([0.1206, 0.243 , 0.1444])

In [53]:
demeaned = arr - arr.mean(0)
demeaned

array([[ 1.6042,  2.3751,  0.633 ],
       [ 0.7081, -1.202 , -1.3538],
       [-1.5329,  0.2985,  0.6076],
       [-0.7793, -1.4717,  0.1132]])

In [54]:
demeaned.mean(0)

array([ 0., -0.,  0.])

<p>See <a href='#broadcasting'>Fig. A.4: Broadcasting over axis 0 with a 1D array</a> for an illustration of this operation. Demeaning the rows as a broadcast operation requires a bit more care. Fortunately, broadcasting potentially lower dimensional values across any dimension of an array (like subtracting the row means from each column of a two-dimensional array) is possible as long as you follow the rules.</p>

<p>This brings us to the broadcasting rule.</p>

<p>Two arrays are compatible for broadcasting if for each <em>trailing dimension</em> (i.e., starting from the end) the axis lengths match or if either of the lengths is 1. Broadcasting is then performed over the missing or length 1 dimensions.</p>
<figure id='broadcasting'>
    <img src="images/pda3_a004.png">
    <figcaption>Fig. A.4: Broadcasting over axis 0 with a 1D array</figcaption>
</figure>
<p>Even as an experienced NumPy user, I often find myself having to pause and draw a diagram as I think about the broadcasting rule. Consider the last example and suppose we wished instead to subtract the mean value from each row. Since <code>arr.mean(0)</code> has length 3, it is compatible for broadcasting across axis 0 because the trailing dimension in <code>arr</code> is 3 and therefore matches. According to the rules, to subtract over axis 1 (i.e., subtract the row mean from each row), the smaller array must have the shape <code>(4, 1)</code>:</p>

In [55]:
arr

array([[ 1.7247,  2.6182,  0.7774],
       [ 0.8286, -0.959 , -1.2094],
       [-1.4123,  0.5415,  0.7519],
       [-0.6588, -1.2287,  0.2576]])

In [56]:
row_means = arr.mean(1)
row_means.shape

(4,)

In [57]:
row_means.reshape((4, 1))

array([[ 1.7068],
       [-0.4466],
       [-0.0396],
       [-0.5433]])

In [58]:
demeaned = arr - row_means.reshape((4, 1))
demeaned.mean(1)

array([-0.,  0.,  0.,  0.])

<p>See <a href='#broadcasting_axis_1'>Fig. A.5: Broadcasting over axis 1 of a 2D array</a> for an illustration of this operation.</p>
<figure id='broadcasting_axis_1'>
    <img src='images/pda3_a005.png'>
    <figcaption>Fig. A.5: Broadcasting over axis 1 of a 2D array</figcaption>
</figure>
    

<p>See <a href='#broadcasting_axis_0'>Fig. A.6: Broadcasting over axis 0 of a 3D array</a> for another illustration, this time adding a two-dimensional array to a three-dimensional one across axis 0.</p>
<figure id='broadcasting_axis_0'>
    <img src="images/pda3_a006.png">
    <figcaption>Fig. A.6: Broadcasting over axis 0 with of a 3D array</figcaption>
</figure>

<h3>Broadcasting over Other Axes</h3>

<p>Broadcasting with higher dimensional arrays can seem even more mind-bending, but it is really a matter of following the rules. If you don’t, you’ll get an error like this:</p>

In [59]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

<p>It’s quite common to want to perform an arithmetic operation with a lower dimensional array across axes other than axis 0. According to the broadcasting rule, the “broadcast dimensions” must be 1 in the smaller array. In the example of row demeaning shown here, this means reshaping the row to be shape <code>(4, 1)</code> instead of <code>(4,)</code>:</p>

In [60]:
arr - arr.mean(1).reshape((4, 1))

array([[ 0.018 ,  0.9114, -0.9294],
       [ 1.2752, -0.5124, -0.7628],
       [-1.3727,  0.5811,  0.7915],
       [-0.1155, -0.6854,  0.8009]])

<p>In the three-dimensional case, broadcasting over any of the three dimensions is only a matter of reshaping the data to be shape compatible. <a href='#compatible'>Fig. A.7: Compatible 2D array shapes for broadcasting over a 3D array</a> nicely visualizes the shapes required to broadcast over each axis of a three-dimensional array.</p>
<figure id='compatible'>
    <img src="images/pda3_a007.png">
    <figcaption>Fig. A.7:Compatible 2D array shapes for broadcasting over a 3D array</figcaption>
</figure>
<p>A common problem, therefore, is needing to add a new axis with length 1 specifically for broadcasting purposes. Using <code>reshape</code> is one option, but inserting an axis requires constructing a tuple indicating the new shape. This often can be a tedious exercise. Thus, NumPy arrays offer a special syntax for inserting new axes by indexing. We use the special <code>np.newaxis</code> attribute along with “full” slices to insert the new axis:</p>

In [61]:
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

(4, 1, 4)

In [62]:
arr_1d = rng.standard_normal(3)
arr_1d[:, np.newaxis]

array([[ 0.3129],
       [-0.1308],
       [ 1.27  ]])

In [63]:
arr_1d[np.newaxis, :]

array([[ 0.3129, -0.1308,  1.27  ]])

<p>Thus, if we had a three-dimensional array and wanted to demean axis 2, we would need to write:</p>

In [64]:
arr = rng.standard_normal((3, 4, 5))
depth_means = arr.mean(2)
depth_means

In [65]:
depth_means.shape

In [66]:
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2)

<p>You might be wondering if there’s a way to generalize demeaning over an axis without sacrificing performance. There is, but it requires some indexing gymnastics:</p>
<pre>
def demean_axis(arr, axis=0):
    means = arr.mean(axis)

    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]
</pre>
<h3>Setting Array values by Broadcasting</h3>
<p>The same broadcasting rule governing arithmetic operations also applies to setting values via array indexing. In a simple case, we can do things like:</p>

In [67]:
arr = np.zeros((4, 3))
arr[:] = 5
arr

<p>However, if we had a one-dimensional array of values we wanted to set into the columns of the array, we can do that as long as the shape is compatible:</p>

In [68]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis]
arr

In [69]:
arr[:2] = [[-1.37], [0.509]]
arr

<h2>A.4 Advanced ufunc Usage</h2>

<p>While many NumPy users will only use the fast element-wise operations provided by the universal functions, a number of additional features occasionally can help you write more concise code without explicit loops.</p>

<h3>ufunc Instance Methods</h3>

<p>Each of NumPy’s binary ufuncs has special methods for performing certain kinds of special vectorized operations. These are summarized in <a href='#ufunc'>Table A.2: ufunc methods</a>, but I’ll give a few concrete examples to illustrate how they work.</p>

<p><code>reduce</code> takes a single array and aggregates its values, optionally along an axis, by performing a sequence of binary operations. For example, an alternative way to sum elements in an array is to use <code>np.add.reduce</code>:</p>

In [70]:
arr = np.arange(10)
np.add.reduce(arr)

In [71]:
arr.sum()

<p>The starting value (for example, 0 for <code>add</code>) depends on the ufunc. If an axis is passed, the reduction is performed along that axis. This allows you to answer certain kinds of questions in a concise way. As a less mundane example, we can use <code>np.logical_and</code> to check whether the values in each row of an array are sorted:</p>

In [72]:
my_rng = np.random.default_rng(12346)  # for reproducibility
arr = my_rng.standard_normal((5, 5))
arr

In [73]:
arr[::2].sort(1) # sort a few rows
arr[:, :-1] < arr[:, 1:]

In [74]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

<p>Note that <code>logical_and</code>.reduce is equivalent to the <code>all</code> method.</p>

<p>The <code>accumulate</code> ufunc method is related to <code>reduce</code>, like <code>cumsum</code> is related to <code>sum</code>. It produces an array of the same size with the intermediate “accumulated” values:</p>

In [75]:
arr = np.arange(15).reshape((3, 5))
np.add.accumulate(arr, axis=1)

<p><code>outer</code> performs a pair-wise cross product between two arrays:</p>

In [76]:
arr = np.arange(3).repeat([1, 2, 2])
arr

In [77]:
np.multiply.outer(arr, np.arange(5))

<p>The output of <code>outer</code> will have a dimension that is the concatenation of the dimensions of the inputs:</p>

In [78]:
x, y = rng.standard_normal((3, 4)), rng.standard_normal(5)
result = np.subtract.outer(x, y)
result.shape

<p>The last method, <code>reduceat</code>, performs a “local reduce,” in essence an array “group by” operation in which slices of the array are aggregated together. It accepts a sequence of “bin edges” that indicate how to split and aggregate the values:</p>

In [79]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])

<p>The results are the reductions (here, sums) performed over <code>arr[0:5]</code>, <code>arr[5:8]</code>, and <code>arr[8:]</code>. As with the other methods, you can pass an axis argument:</p>

In [80]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr

In [81]:
np.add.reduceat(arr, [0, 2, 4], axis=1)

<p>See <a href='#ufunc'>Table A.2: ufunc methods</a> for a partial listing of ufunc methods.</p>
<table id='ufunc'>
    <caption>Table A.2: ufunc methods</caption>
    <tr>
        <th>Method</th>
        <th>Description</th>
    </tr>
    <tr>
        <td><code>accumulate(x)</code></td>
        <td>Aggregate values, preserving all partial aggregates.</td>
    </tr>
    <tr>
        <td><code>at(x, indieces, b=None)</code></td>
        <td>Perform operation in place on <code>x</code> at the specified indices. The argument <code>b</code> is the second input to ufuncs that requires two array inputs.</td>
    </tr>
    <tr>
        <td><code>reduce(x)</code></td>
        <td>Aggregate values by successive applications of the operation.</td>
    </tr>
    <tr>
        <td><code>reduceat(x, bins)</code></td>
        <td>“Local” reduce or “group by”; reduce contiguous slices of data to produce an aggregated array.</td>
    </tr>
    <tr>
        <td><code>outer(x, y)</code></td>
        <td>Apply operation to all pairs of elements in <code>x</code> and <code>y</code>; the resulting array has shape <code>x.shape + y.shape</code>.</td>
    </tr>
</table>
<h3>Writing New ufunc in Python</h3>
<p>There are a number of ways to create your own NumPy ufuncs. The most general is to use the NumPy C API, but that is beyond the scope of this book. In this section, we will look at pure Python ufuncs.</p>

<p><code>numpy.frompyfunc</code> accepts a Python function along with a specification for the number of inputs and outputs. For example, a simple function that adds element-wise would be specified as:</p>

In [82]:
def add_elements(x, y):
    return x + y
add_them = np.frompyfunc(add_elements, 2, 1)
add_them(np.arange(8), np.arange(8))

<p>Functions created using <code>frompyfunc</code> always return arrays of Python objects, which can be inconvenient. Fortunately, there is an alternative (but slightly less feature rich) function, <code>numpy.vectorize</code>, that allows you to specify the output type:</p>

In [83]:
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))

<p>These functions provide a way to create ufunc-like functions, but they are very slow because they require a Python function call to compute each element, which is a lot slower than NumPy’s C-based ufunc loops:</p>

In [147]:
arr = rng.standard_normal(10000)
%timeit add_them(arr, arr)
%timeit np.add(arr, arr)

<p>Later in this appendix we'll show how to create fast ufuncs in Python using the <a href="http://numba.pydata.org/">Numba library</a>.
    
<h2>A.5 Structured and Record Arrays</h2>
<span id='a_5'></span>
<p>You may have noticed up until now that ndarray is a <em>homogeneous</em> data container; that is, it represents a block of memory in which each element takes up the same number of bytes, as determined by the data type. On the surface, this would appear to not allow you to represent heterogeneous or tabular data. A <em>structured</em> array is an ndarray in which each element can be thought of as representing a <em>struct</em> in C (hence the “structured” name) or a row in a SQL table with multiple named fields:</p>

In [85]:
dtype = [('x', np.float64), ('y', np.int32)]
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype)
sarr

<p>There are several ways to specify a structured data type (see the online NumPy documentation). One typical way is as a list of tuples with <code>(field_name, field_data_type)</code>. Now, the elements of the array are tuple-like objects whose elements can be accessed like a dictionary:</p>

In [86]:
sarr[0]

In [87]:
sarr[0]['y']

<p>The field names are stored in the <code>dtype.names</code> attribute. When you access a field on the structured array, a strided view on the data is returned, thus copying nothing:</p>

In [88]:
sarr['x']

<h3>Nested Data Types and Multidimensional Fields</h3>

<p>When specifying a structured data type, you can additionally pass a shape (as an int or tuple):</p>

In [89]:
dtype = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype=dtype)
arr

<p>In this case, the <code>x</code> field now refers to an array of length 3 for each record:</p>

In [90]:
arr[0]['x']

<p>Conveniently, accessing <code>arr['x']</code> then returns a two-dimensional array instead of a one-dimensional array as in prior examples:</p>

In [91]:
arr['x']

<p>This enables you to express more complicated, nested structures as a single block of memory in an array. You can also nest data types to make more complex structures. Here is an example:</p>

In [92]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)
data['x']

In [93]:
data['y']

In [94]:
data['x']['a']

<p>pandas DataFrame does not support this feature in the same way, though it is similar to hierarchical indexing.</p>

<h3>Why Use Structured Arrays?</h3>

<p>Compared with a pandas DataFrame, NumPy structured arrays are a lower level tool. They provide a means to interpret a block of memory as a tabular structure with nested columns. Since each element in the array is represented in memory as a fixed number of bytes, structured arrays provide an efficient way of writing data to and from disk (including memory maps), transporting it over the network, and other such uses. The memory layout of each value in a structured array is based on the binary representation of struct data types in the C programming language.</p>

<p>As another common use for structured arrays, writing data files as fixed-length record byte streams is a common way to serialize data in C and C++ code, which is sometimes found in legacy systems in industry. As long as the format of the file is known (the size of each record and the order, byte size, and data type of each element), the data can be read into memory with <code>np.fromfile</code>. Specialized uses like this are beyond the scope of this book, but it’s worth knowing that such things are possible.</p>

<h2>A.6 More About Sorting</h2>

<p>Like Python’s built-in list, the ndarray <code>sort</code> instance method is an in-place sort, meaning that the array contents are rearranged without producing a new array:</p>

In [95]:
arr = rng.standard_normal(6)
arr.sort()
arr

<p>When sorting arrays in place, remember that if the array is a view on a different ndarray, the original array will be modified:</p>

In [96]:
arr = rng.standard_normal((3, 5))
arr

In [97]:
arr[:, 1].sort()  # Sort second column values in place
arr

<p>On the other hand, <code>numpy.sort</code> creates a new, sorted copy of an array. Otherwise, it accepts the same arguments (such as <code>kind</code>) as ndarray's <code>sort</code> method:</p>

In [98]:
arr = rng.standard_normal(5)
arr

In [99]:
np.sort(arr)

In [100]:
arr

<p>All of these sort methods take an axis argument for independently sorting the sections of data along the passed axis:</p>

In [101]:
arr = rng.standard_normal((3, 5))
arr

In [102]:
arr.sort(axis=1) # sort by row
arr

<p>You may notice that none of the sort methods have an option to sort in descending order. This is a problem in practice because array slicing produces views, thus not producing a copy or requiring any computational work. Many Python users are familiar with the “trick” that for a list of <code>values</code>, <code>values[::-1]</code> returns a list in reverse order. The same is true for ndarrays:</p>

In [103]:
arr[:, ::-1]

<h3>Indirect Sorts: <code>argsort</code> and <code>lexsort</code></h3>

<p>In data analysis you may need to reorder datasets by one or more keys. For example, a table of data about some students might need to be sorted by last name, then by first name. This is an example of an indirect sort, and if you’ve read the pandas-related chapters, you have already seen many higher-level examples. Given a key or keys (an array of values or multiple arrays of values), you wish to obtain an array of integer indices (I refer to them colloquially as indexers) that tells you how to reorder the data to be in sorted order. Two methods for this are <code>argsort</code> and <code>numpy.lexsort</code>. As an example:</p>

In [104]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()
indexer

In [105]:
values[indexer]

<p>As a more complicated example, this code reorders a two-dimensional array by its first row:</p>

In [106]:
arr = rng.standard_normal((3, 5))
arr[0] = values
arr

In [107]:
arr[:, arr[0].argsort()]

<p><code>lexsort</code> is similar to <code>argsort</code>, but it performs an indirect <em>lexicographical</em> sort on multiple key arrays. Suppose we wanted to sort some data identified by first and last names:</p>

In [108]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
sorter

In [109]:
list(zip(last_name[sorter], first_name[sorter]))

<p><code>lexsort</code> can be a bit confusing the first time you use it, because the order in which the keys are used to order the data starts with the <em>first</em> array passed. Here, <code>last_name</code> was used before <code>first_name</code>.</p>

<h3>Alternative Sort Algorithms</h3>

<p>A <em>stable</em> sorting algorithm preserves the relative position of equal elements. This can be especially important in indirect sorts where the relative ordering is meaningful:</p>

In [110]:
values = np.array(['2:first', '2:second', '1:first', '1:second',
                   '1:third'])
key = np.array([2, 2, 1, 1, 1])
indexer = key.argsort(kind='mergesort')
indexer

In [111]:
values.take(indexer)

<p>The only stable sort available is mergesort, which has guaranteed <code>O(n log n)</code> performance, but its performance is on average worse than the default quicksort method. See <a href='#array_sorting'>Table A.3: Array sorting methods</a> for a summary of available methods and their relative performance (and performance guarantees). This is not something that most users will ever have to think about, but it's useful to know that it’s there.</p>

<table id="array_sorting">
    <caption>Table A.3: Array sorting methods</caption>
    <tr>
        <th>Kind</th>
        <th>Speed</th>
        <th>Stable</th>
        <th>Work space</th>
        <th>Worst case</th>
    </tr>
    <tr>
        <td><code>'quicksort'</code></td>
        <td>1</td>
        <td>No</td>
        <td>0</td>
        <td><code>O(n^2)</code></td>
    </tr>
    <tr>
        <td><code>'mergesort'</code></td>
        <td>2</td>
        <td>Yes</td>
        <td><code>O(n / 2)</code></td>
        <td><code>O(n log n)</code></td>
    </tr>
    <tr>
        <td><code>3</code></td>
        <td>3</td>
        <td>No</td>
        <td>O</td>
        <td><code>O(n log n)</code></td>
    </tr>
</table>

<h3>Partially Sorting Arrays</h3>

<p>One of the goals of sorting can be to determine the largest or smallest elements in an array. NumPy has fast methods, <code>numpy.partition</code> and <code>np.argpartition</code>, for partitioning an array around the <code>k</code>-th smallest element:</p>

In [112]:
rng = np.random.default_rng(12345)
arr = rng.standard_normal(20)
arr

In [113]:
np.partition(arr, 3)

<p>After you call <code>partition(arr, 3)</code>, the first three elements in the result are the smallest three values in no particular order. <code>numpy.argpartition</code>, similar to <code>numpy.argsort</code>, returns the indices that rearrange the data into the equivalent order:</p>

In [114]:
indices = np.argpartition(arr, 3)
indices

In [115]:
arr.take(indices)

<h3><code>numpy.searchsorted</code>: Finding Elements in a Sorted Array</h3>

<p><code>searchsorted</code> is an array method that performs a binary search on a sorted array, returning the location in the array where the value would need to be inserted to maintain sortedness:</p>

In [116]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

<p>You can also pass an array of values to get an array of indices back:</p>

In [117]:
arr.searchsorted([0, 8, 11, 16])

<p>You might have noticed that <code>searchsorted</code> returned <code>0</code> for the <code>0</code> element. This is because the default behavior is to return the index at the left side of a group of equal values:</p>

In [118]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1])

In [119]:
arr.searchsorted([0, 1], side='right')

<p>As another application of <code>searchsorted</code>, suppose we had an array of values between 0 and 10,000, and a separate array of “bucket edges” that we wanted to use to bin the data:</p>

In [120]:
data = np.floor(rng.uniform(0, 10000, size=50))
bins = np.array([0, 100, 1000, 5000, 10000])
data

<p>To then get a labeling to which interval each data point belongs (where 1 would mean the bucket <code>[0, 100)</code>), we can simply use <code>searchsorted</code>:</p>

In [121]:
labels = bins.searchsorted(data)
labels

<p>This, combined with pandas’s <code>groupby</code>, can be used to bin data:</p>

In [122]:
pd.Series(data).groupby(labels).mean()

<h2>A.7 Writing Fast NumPy Functions with Numba</h2>

<p><a href='http://numba.pydata.org/'>Numba</a> is an open source project that creates fast functions for NumPy-like data using CPUs, GPUs, or other hardware. It uses the <a href='http://llvm.org/'>LLVM Project</a> to translate Python code into compiled machine code.</p>

<p>To introduce Numba, let's consider a pure Python function that computes the expression <code>(x - y).mean()</code> using a <code>for</code> loop:</p>

In [123]:
import numpy as np

def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

<p>This function is slow:</p>

In [124]:
x = rng.standard_normal(10_000_000)
y = rng.standard_normal(10_000_000)
%timeit mean_distance(x, y)
%timeit (x - y).mean()

<p>The NumPy version is over 100 times faster. We can turn this function into a compiled Numba function using the <code>numba.jit</code> function:</p>

In [125]:
import numba as nb
numba_mean_distance = nb.jit(mean_distance)

<p>We also could have written this as a decorator:</p>

In [126]:
@nb.jit
def numba_mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

<p>The resulting function is actually faster than the vectorized NumPy version:</p>

In [127]:
%timeit numba_mean_distance(x, y)

<p>Numba cannot compile all pure Python code, but it supports a significant subset of Python that is most useful for writing numerical algorithms.</p>

<p>Numba is a deep library, supporting different kinds of hardware, modes of compilation, and user extensions. It is also able to compile a substantial subset of the NumPy Python API without explicit <code>for</code> loops. Numba is able to recognize constructs that can be compiled to machine code, while substituting calls to the CPython API for functions that it does not know how to compile. Numba's <code>jit</code> function option, <code>nopython=True</code>, restricts allowed code to Python code that can be compiled to LLVM without any Python C API calls. <code>jit(nopython=True)</code> has a shorter alias, <code>numba.njit</code>.

<p>In the previous example, we could have written:</p>

In [128]:
from numba import float64, njit

@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

<p>I encourage you to learn more by reading the <a href='http://numba.pydata.org/'>online documentation for Numba</a>. The next section shows an example of creating custom NumPy ufunc objects.</p>

<h3>Creating Custom <code>numpy.ufunc</code> Objects with Numba</h3>

The <code>numba.vectorize</code> function creates compiled NumPy ufuncs, which behave like built-in ufuncs. Let's consider a Python implementation of <code>numpy.add</code>:</p>

In [129]:
from numba import vectorize

@vectorize
def nb_add(x, y):
    return x + y

<p>Now we have:</p>

In [130]:
x = np.arange(10)
nb_add(x, x)

In [131]:
nb_add.accumulate(x, 0)

<h2>A.8 Advanced Array Input and Output</h2>

<p>In Ch 4: NumPy Basics: Arrays and Vectorized Computation, we became acquainted with <code>np.save</code> and <code>np.load</code> for storing arrays in binary format on disk. There are a number of additional options to consider for more sophisticated use. In particular, memory maps have the additional benefit of enabling you to do certain operations with datasets that do not fit into RAM.</p>

<h3>Memory-Mapped Files</h3>

<p>A <em>memory-mapped</em> file is a method for interacting with binary data on disk as though it is stored in an in-memory array. NumPy implements a <code>memmap</code> object that is ndarray-like, enabling small segments of a large file to be read and written without reading the whole array into memory. Additionally, a <code>memmap</code> has the same methods as an in-memory array and thus can be substituted into many algorithms where an ndarray would be expected.</p>

<p>To create a new memory map, use the function <code>np.memmap</code> and pass a file path, data type, shape, and file mode:</p>

In [132]:
mmap = np.memmap('mymmap', dtype='float64', mode='w+',
                 shape=(10000, 10000))
mmap

<p>Slicing a <code>memmap</code> returns views on the data on disk:</p>

In [133]:
section = mmap[:5]

<p>If you assign data to these, it will be buffered in memory, which means that the changes will not be immediately reflected in the on-disk file if you were to read the file in a different application. Any modifications can be synchronized to disk by calling <code>flush</code>:</p>

In [134]:
section[:] = rng.standard_normal((5, 10000))
mmap.flush()
mmap

In [135]:
del mmap

<p>Whenever a memory map falls out of scope and is garbage collected, any changes will be flushed to disk also. When <em>opening an existing memory map</em>, you still have to specify the data type and shape, as the file is only a block of binary data without any data type information, shape, or strides:</p>

In [136]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))
mmap

<p>Memory maps also work with structured or nested data types, as described in <a href='#a_5'>A.5 Structured and Record Arrays</a>.</p>
<p>If you ran this example on your computer, you may want to delete the large file that we created above:</p>

In [137]:
%xdel mmap
!rm mymmap

<h3>HDF5 and Other Array Storage Options</h3>

<p>PyTables and h5py are two Python projects providing NumPy-friendly interfaces for storing array data in the efficient and compressible HDF5 format (HDF stands for <em>hierarchical data format</em>). You can safely store hundreds of gigabytes or even terabytes of data in HDF5 format. To learn more about using HDF5 with Python, I recommend reading the <a href='http://pandas.pydata.org/'>pandas online documentation</a>.</p>

<h2>A.9 Performance Tips</h2>

<p>Adapting data processing code to use NumPy generally makes things much faster, as array operations typically replace otherwise comparatively extremely slow pure Python loops. Here are some tips to help get the best performance out of the library:</p>
<ul>
    <li>Convert Python loops and conditional logic to array operations and Boolean array operations.</li>
    <li>Use broadcasting whenever possible.</li>
    <li>Use arrays views (slicing) to avoid copying data.</li>
    <li>Utilize ufuncs and ufunc methods.</li>
</ul>

<p>If you can’t get the performance you require after exhausting the capabilities provided by NumPy alone, consider writing code in C, FORTRAN, or Cython. I use <a href='http://cython.org/'>Cython</a> frequently in my own work as a way to get C-like performance, often with much less development time.</p>

<h3>The Importance of Contiguous Memory</h3>

<p>While the full extent of this topic is a bit outside the scope of this book, in some applications the memory layout of an array can significantly affect the speed of computations. This is based partly on performance differences having to do with the cache hierarchy of the CPU; operations accessing contiguous blocks of memory (e.g., summing the rows of a C order array) will generally be the fastest because the memory subsystem will buffer the appropriate blocks of memory into the low latency L1 or L2 CPU caches. Also, certain code paths inside NumPy’s C codebase have been optimized for the contiguous case in which generic strided memory access can be avoided.</p>

<p>To say that an array’s memory layout is contiguous means that the elements are stored in memory in the order that they appear in the array with respect to FORTRAN (column major) or C (row major) ordering. By default, NumPy arrays are created as C contiguous or just simply contiguous. A column major array, such as the transpose of a C-contiguous array, is thus said to be FORTRAN contiguous. These properties can be explicitly checked via the <code>flags</code> attribute on the ndarray:</p>

In [138]:
arr_c = np.ones((100, 10000), order='C')
arr_f = np.ones((100, 10000), order='F')
arr_c.flags

In [139]:
arr_f.flags

In [140]:
arr_f.flags.f_contiguous

<p>In this example, summing the rows of these arrays should, in theory, be faster for <code>arr_c</code> than <code>arr_f</code> since the rows are contiguous in memory. Here, I check using <code>%timeit</code> in IPython (these results may differ on your machine):</p>

In [141]:
%timeit arr_c.sum(1)
%timeit arr_f.sum(1)

<p>When you're looking to squeeze more performance out of NumPy, this is often a place to invest some effort. If you have an array that does not have the desired memory order, you can use <code>copy</code> and pass either <code>'C'</code> or <code>'F'</code>:</p>

In [142]:
arr_f.copy('C').flags

<p>When constructing a view on an array, keep in mind that the result is not guaranteed to be contiguous:</p>

In [143]:
arr_c[:50].flags.contiguous

In [144]:
arr_c[:, :50].flags

<p>If you ran this example on your computer, you may want to delete the large objects that we created above:</p>

In [145]:
%xdel arr_c
%xdel arr_f

In [146]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS